# Fit the Model to Training Data

## Setup

In [1]:
from importlib import reload

import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt

from hot_dust import preprocess, model

2024-08-15 15:17:40.675165: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-15 15:17:40.716765: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-15 15:17:40.716800: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-15 15:17:40.717513: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-15 15:17:40.723099: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-15 15:17:40.723568: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
# "reload" to get changes in preprocess.py without restarting the kernel
reload(preprocess)
reload(model)
from hot_dust.preprocess import prepare_training_data, split_training_data
from hot_dust.model import to_tensorflow, compile, pretraining

### Parameters

In [3]:
epochs = 200  # Max 500
batch_size = 64
buffer_size = 10 * batch_size

ds = prepare_training_data()
train, validate, test = to_tensorflow(split_training_data(ds))

train = train.shuffle(buffer_size).batch(batch_size)
validate = validate.batch(batch_size).cache()

In [4]:
layer = pretraining(dataset=test)
network = compile(normalization=layer)
network.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9)]               0         
                                                                 
 normalization (Normalizati  (None, 9)                 19        
 on)                                                             
                                                                 
 dense (Dense)               (None, 16)                160       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 468 (1.83 KB)
Trainable params: 449 (1.75 KB)
Non-trainable params: 19 (80.00 Byte)
_____________________________

### Fitting

In [5]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=20,
    verbose=1,
)
# List for callbacks
callbacks_list = [early_stopping]

fit = network.fit(
    train,
    epochs=epochs,
    validation_data=validate,
    verbose=2,
    callbacks=[early_stopping],
)

network.save("data/network")
fit = {"epoch": fit.epoch, **fit.history}
np.savez("data/fit.npz", **fit)

Epoch 1/200
3122/3122 - 4s - loss: 0.5138 - val_loss: 0.3349 - 4s/epoch - 1ms/step
Epoch 2/200
3122/3122 - 3s - loss: 0.2946 - val_loss: 0.2692 - 3s/epoch - 1ms/step
Epoch 3/200
3122/3122 - 3s - loss: 0.2458 - val_loss: 0.2283 - 3s/epoch - 1ms/step
Epoch 4/200
3122/3122 - 3s - loss: 0.2114 - val_loss: 0.1994 - 3s/epoch - 1ms/step
Epoch 5/200
3122/3122 - 3s - loss: 0.1848 - val_loss: 0.1775 - 3s/epoch - 1ms/step
Epoch 6/200
3122/3122 - 3s - loss: 0.1657 - val_loss: 0.1600 - 3s/epoch - 1ms/step
Epoch 7/200
3122/3122 - 3s - loss: 0.1517 - val_loss: 0.1466 - 3s/epoch - 1ms/step
Epoch 8/200
3122/3122 - 3s - loss: 0.1408 - val_loss: 0.1366 - 3s/epoch - 1ms/step
Epoch 9/200
3122/3122 - 3s - loss: 0.1320 - val_loss: 0.1284 - 3s/epoch - 1ms/step
Epoch 10/200
3122/3122 - 3s - loss: 0.1251 - val_loss: 0.1216 - 3s/epoch - 1ms/step
Epoch 11/200
3122/3122 - 3s - loss: 0.1194 - val_loss: 0.1182 - 3s/epoch - 1ms/step
Epoch 12/200
3122/3122 - 3s - loss: 0.1149 - val_loss: 0.1130 - 3s/epoch - 1ms/step
E

INFO:tensorflow:Assets written to: data/network/assets


### Training and Validation Losses

In [8]:
fit = np.load("data/fit.npz")

In [9]:
model.plot_loss(fit)

:NdOverlay   [split]
   :Scatter   [epoch]   (value)